In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip3 install ipython-autotime

%load_ext autotime


import pandas as pd
import numpy as np
# import pickle5 as pickle
import re
from nltk.stem import WordNetLemmatizer
import nltk
from tqdm.notebook import tqdm
tqdm.pandas()
import requests

from termcolor import colored
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

nb_workers=10

import os
from datetime import datetime
# import pickle
import ast
import re
my_dir = '/content/drive/MyDrive/Discord-Reddit/'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 15.3 MB/s 
time: 1.82 s (started: 2022-12-01 19:26:24 +00:00)


In [ ]:
import os
discords = os.listdir('/content/drive/MyDrive/Discord-Reddit/Datasets/discord')
subreddits = os.listdir('/content/drive/MyDrive/Discord-Reddit/Datasets/reddit')

# discords.remove('.ipynb_checkpoints')
# subreddits.remove('.ipynb_checkpoints')

time: 4.92 ms (started: 2022-11-29 17:27:44 +00:00)


In [3]:
def clean_df(data,text_col,author_col):
  data[text_col]=data[text_col].astype(str)
  data=data[data[text_col].map(len) > 2]
  data=data[data[author_col]!='error']
  data=data[data[author_col]!=0]
  data=data[data [author_col]!='AutoModerator']
  data.drop_duplicates(subset=text_col, keep='first', inplace=True, ignore_index=False)
  return data

time: 6.05 ms (started: 2022-12-01 19:28:27 +00:00)


In [4]:
def get_average_toxicity(community_name_x, agg_x='mean'):

  data_reddit=pd.read_csv(f'{my_dir}Datasets/reddit/{community_name_x}_subreddit_hate.csv', lineterminator='\n')
  data_discord=pd.read_csv(f'{my_dir}Datasets/discord/{community_name_x}_discord_hate.csv', lineterminator='\n')

  data_reddit=clean_df(data_reddit,text_col='comment_text',author_col='author')
  data_discord=clean_df(data_discord,text_col='content',author_col='author_username')

  return {'community':community_name_x,'reddit_toxicity':data_reddit['Toxic_BERT'].mean(),'discord_toxicity':data_discord['Toxic_BERT'].mean()}


time: 1.63 ms (started: 2022-12-01 19:33:16 +00:00)


In [6]:
community_names = ['Overwatch','europe','Jokes','nosleep','Rainbow6','history','Ukrainian','ksi','sports',
 'dankmemes','Games','kpop','music','rickandmorty','Writingprompts']
community_toxicity_df = pd.DataFrame()
for community_name_i in tqdm(community_names):
  print(community_name_i)
  community_toxicity_df=community_toxicity_df.append(get_average_toxicity(community_name_x = community_name_i), ignore_index=True)

  0%|          | 0/15 [00:00<?, ?it/s]

Overwatch
europe
Jokes
nosleep
Rainbow6
history
Ukrainian


<ipython-input-6-4feeff04efae>:6: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  community_toxicity_df=community_toxicity_df.append(get_average_toxicity(community_name_x = community_name_i), ignore_index=True)


ksi
sports
dankmemes
Games
kpop
music
rickandmorty
Writingprompts
time: 3min 19s (started: 2022-12-01 19:37:38 +00:00)


In [17]:
community_toxicity_df.to_excel(f'{my_dir}Datasets/platform_toxicity_average.xlsx')

time: 335 ms (started: 2022-12-01 20:07:30 +00:00)


In [14]:
pd.read_csv(f'{my_dir}Datasets/discord/Ukrainian_discord_hate.csv', lineterminator='\n')

,community,reddit_toxicity,discord_toxicity
0,Overwatch,0.098617,0.104188
1,europe,0.103206,0.103064
2,Jokes,0.148267,0.080039
3,nosleep,0.120727,0.058809
4,Rainbow6,0.113228,0.087537
5,history,0.023065,0.022049
6,Ukrainian,0.146105,0.076104
7,ksi,0.177265,0.142110
8,sports,0.142134,0.062544
9,dankmemes,0.181769,0.108872


time: 9.02 ms (started: 2022-12-01 19:50:52 +00:00)


In [16]:
def cutoff(float_x):
  if float_x > 0.5:
    return 1
  if float_x <= 0.5:
    return 0
data_reddit['Toxic_BERT'].apply(lambda x: cutoff(float_x=x)).mean()

0.06252546207795627

time: 159 ms (started: 2022-12-01 19:51:33 +00:00)
